In [1]:
import os
import requests
import json
from dotenv import load_dotenv
from openai import OpenAI

In [2]:
load_dotenv()
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
AIDEVS_API_KEY = os.environ["AIDEVS_API_KEY"]

In [3]:
client = OpenAI()

In [4]:
response = requests.post("https://tasks.aidevs.pl/token/liar", json=dict(apikey=AIDEVS_API_KEY))
response_json = json.loads(response.text)
token = response_json["token"]

response = requests.get(f"https://tasks.aidevs.pl/task/{token}")
response_json = json.loads(response.text)

print(response_json)

{'code': 0, 'msg': 'send me any question in english, and I will try to answer it in max 150 tokens', 'hint1': "please send your question in 'question' field to /task/ endpoint (simple form, not JSON)", 'hint2': "sometimes I don't tell the truth", 'hint3': "Send to /answer/ info if I'm telling the truth. Just value: YES/NO"}


In [5]:
question = "What is the title of the pizza Margherita?"

response = requests.post(f"https://tasks.aidevs.pl/task/{token}", data={"question": question})
response_json = json.loads(response.text)
doubt_answer = response_json["answer"]

print(response_json)

{'code': 0, 'msg': 'This is my answer', 'answer': 'Steven Spielberg, who directed the film, came up with the idea for the vibrating water glass while listening to Earth, Wind & Fire in his car.'}


In [6]:
system_content = """
You are a Guardrails systems that checks if the answer for given question makes any sense or not. You can return either YES - if you think the answer is correct - or NO if you don't

Examples:
question: What is capital of Poland?
answer: There are plantny restaurants in Rome serving best pizza in the world.

asistant: NO
"""

In [7]:
completion = client.chat.completions.create(
  model="gpt-4",
  messages=[
    {"role": "system", "content": system_content},
    {"role": "user", "content": f"question: {question}\n answer: {doubt_answer}"}
  ]
)

asistant_validation = str(completion.choices[0].message.content)
print(asistant_validation)

NO


In [8]:
response = requests.post(f"https://tasks.aidevs.pl/answer/{token}", json=dict(answer=asistant_validation))
print(response.text)

{
    "code": 0,
    "msg": "OK",
    "note": "CORRECT"
}
